In [ ]:
import os

import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize

import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union

import numpy as np
import cv2
import matplotlib.pyplot as plt

### Split Image by pixel size

In [ ]:
import os
from osgeo import gdal

# Put image directory and name for split
in_path = 'C:/Users/scsi/Desktop/SIGspatial/0.image_process/1.image/0619/'
input_filename = '0619_3.tif' 

# Save file 
out_path = 'D:/split/0619'
output_filename = '0619_3_'

# Size of image splited
tile_size_x = 256 
tile_size_y = 256
overlap = 128

ds = gdal.Open(os.path.join(in_path, input_filename))
band = ds.GetRasterBand(1)
xsize = band.XSize
ysize = band.YSize

for i in range(0, xsize, tile_size_x - overlap):
    for j in range(0, ysize, tile_size_y - overlap):
        width = min(tile_size_x, xsize - i)  # Adjust for remaining space in x direction
        height = min(tile_size_y, ysize - j)  # Adjust for remaining space in y direction
        com_string = (
            f"gdal_translate -of GTIFF -srcwin {i}, {j}, {width}, {height} "
            f"{os.path.join(in_path, input_filename)} "
            f"{os.path.join(out_path, output_filename)}{i}_{j}.tif"
        )
        os.system(com_string)

### Only 256*256

In [ ]:
from PIL import Image
import os

def is_image_256x256(file_path):
    try:
        with Image.open(file_path) as img:
            return img.size == (256, 256)
    except:
        return False

def delete_non_256x256_tiff_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".tiff") or filename.lower().endswith(".tif"):
            file_path = os.path.join(folder_path, filename)
            if not is_image_256x256(file_path):
                os.remove(file_path)
                print(f"Deleted: {filename}")

folder_path = r"D:\split\0619"
delete_non_256x256_tiff_files(folder_path)

### Make Mask(mask = 0)

In [ ]:
def poly_from_utm(polygon, transform):
    poly_pts = []
    
    poly = cascaded_union(polygon)
    for i in np.array(poly.exterior.coords):
        # Convert polygons to the image CRS
        poly_pts.append(~transform * tuple(i))
        
    # Generate a polygon object
    new_poly = Polygon(poly_pts)
    return new_poly


raster_folder = r"D:\split\0619"
output_folder = r"D:\split\0619_mask"
for raster_filename in os.listdir(raster_folder):
    if raster_filename.endswith('.tif'):
        raster_path = os.path.join(raster_folder, raster_filename)
        
        with rasterio.open(raster_path, "r") as src:
            raster_img = src.read()
            raster_meta = src.meta

        shape_path = r"D:\split\06119_mask.shp"  # Mask shapefile
        train_df = gpd.read_file(shape_path)

        poly_shp = []
        im_size = (src.meta['height'], src.meta['width'])
        for num, row in train_df.iterrows():
            if row['geometry'].geom_type == 'Polygon':
                poly = poly_from_utm(row['geometry'], src.meta['transform'])
                poly_shp.append(poly)
            else:
                for p in row['geometry']:
                    poly = poly_from_utm(p, src.meta['transform'])
                    poly_shp.append(poly)

        mask = rasterize(shapes=poly_shp, out_shape=im_size)

        mask = mask.astype("uint16")
        
        inverted_mask = 1 - mask   ## make mask = 0
        
        save_filename = f"{os.path.splitext(raster_filename)[0]}.tif"
        save_path = os.path.join(output_folder, save_filename)
        
        bin_mask_meta = src.meta.copy()
        bin_mask_meta.update({'count': 1})
        with rasterio.open(save_path, 'w', **bin_mask_meta) as dst:
            dst.write(inverted_mask * 255, 1) 
print("Processing complete.")